In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'c:\\Michal\\Programming\\Repositories_MG\\End-to-End-ML-project'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [4]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories
import os

In [5]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_checkpoint_dir = Path(config.checkpoint_model_filepath).parent
        create_directories([
            Path(model_checkpoint_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [6]:
import os 
from pathlib import Path
import tensorflow as tf
import time

In [22]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = self.config.tensorboard_root_log_dir / f"tb_logs_at_{timestamp}"
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_checkpoint_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_checkpoint_callback(self):
        return [self._create_tb_callbacks,
                self._create_checkpoint_callbacks]

In [23]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_checkpoint_callback()
except Exception as e:
    raise e

[2023-10-07 14:16:45,574: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-07 14:16:45,575: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-07 14:16:45,576: INFO: common: created directory at: artifacts]
[2023-10-07 14:16:45,577: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-10-07 14:16:45,578: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
